In [5]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
import os
import openai

In [4]:

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# memory = ConversationBufferWindowMemory( k=2, 
#                                         memory_key='chat_history',
#                                         return_docs=False,
#                                         verbose = True)

In [7]:
conversation = ConversationChain(
    llm=OpenAI(temperature=0,  openai_api_key=openai.api_key), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [8]:
conversation.predict(input="I'm good! I'm having issue in internet connection")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: I'm good! I'm having issue in internet connection
AI:

> Finished chain.


' Oh no, that sounds like a frustrating issue. What kind of internet connection do you have?'

In [9]:
conversation.predict(input = "could you help me with that?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: I'm good! I'm having issue in internet connection
AI:  Oh no, that sounds like a frustrating issue. What kind of internet connection do you have?
Human: could you help me with that?
AI:

> Finished chain.


" I'm sorry, I'm not sure I can help you with that. I'm not an expert in internet connections. However, I can provide you with some resources that might be able to help. Would that be helpful?"

In [50]:
def retrieval():
    print("answer retrieved")

In [51]:
from langchain.chains.router.llm_router import LLMRouterChain
general_template = """
                    
                    If the following input is not related to Canadian immigration
                    reply that you are designed to answer the questions on Canadian immigration.
                    otherwise call retrieval() function
                    {input}
                    """


greeting_template = """
                    You are a virtual assistant who can greet and thank customers based on below input.
                    {input}
                    """

In [52]:
prompt_infos = [
    {
        "name": "general",
        "description": "Good for answering questions about immigration",
        "prompt_template": general_template,
    },
    {
        "name": "greeting",
        "description": "Good for answering greeting/thank you",
        "prompt_template": greeting_template,
    },
]

In [53]:
llm = OpenAI()

In [54]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain


In [55]:
default_chain = ConversationChain(llm=llm, output_key="text")

In [56]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [57]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
print(destinations)
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print("===")
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

['general: Good for answering questions about immigration', 'greeting: Good for answering greeting/thank you']
===


In [58]:
from langchain.chains.router import MultiPromptChain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [59]:
print(chain.run("hey, How are You?"))



> Entering new MultiPromptChain chain...


d:\Divya\Job\Volunteering\Omdena_Projects\Toronto_Chapter\chatbot_venv\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


greeting: {'input': 'Hey, how are you?'}
> Finished chain.

                    Hi there! I'm doing great, how about yourself?


In [61]:
print(chain.run("What is the procedure to apply PR in canada"))



> Entering new MultiPromptChain chain...
general: {'input': 'What is the procedure to apply for permanent residence in Canada?'}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



> Finished chain.

                    The procedure to apply for permanent residence in Canada depends on the immigration program you choose. Generally, you must first create an account with Immigration, Refugees and Citizenship Canada (IRCC). Then, you must fill out the necessary forms and submit the required documents. You may also need to provide biometrics and a language test result. Once your application is complete, you must pay the fees and submit your application. IRCC will then review your application and make a decision.


In [62]:
print(chain.run("What is the procedure to apply green card"))



> Entering new MultiPromptChain chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

general: {'input': 'What is the procedure to apply for a green card?'}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


> Finished chain.

                   I am designed to answer the questions on Canadian immigration.


In [48]:
print(chain.run("Bye"))



> Entering new MultiPromptChain chain...
greeting: {'input': 'Bye'}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UiDO1YtUleB9WfvC2KZVSADs on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


> Finished chain.

                    Thanks for stopping by! Have a great day!
